In [21]:
import sys
import os
if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount("/content/drive")
    %cd /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks

    dname = "nb009-cv"
    os.makedirs(dname, exist_ok=True)


    !pip install --upgrade --force-reinstall --no-deps kaggle
    import json
    f = open("/content/drive/MyDrive/colab_notebooks/kaggle/kaggle.json", "r")
    json_data = json.load(f)
    os.environ["KAGGLE_USERNAME"] = json_data["username"]
    os.environ["KAGGLE_KEY"] = json_data["key"]


    import yaml
    from pprint import pprint
    with open('../config/config.yml') as file:
        CFG = yaml.load(file)
    pprint(CFG)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks
     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=a43d6958e4e848496d3dce4d16e95d7f49b454102c79a54d71776d8000559922
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
{'debug': False,
 'epochs': 5,
 'is_single': False,
 'learning_rate': 2e-05,
 'max_len': 512,
 'tags_vals': 'o o-dataset pad',
 'test_batch_size': 48,
 'text_len': 0,
 'train': True,
 'train_batch_size': 32,
 'use_cosine': False,
 'use_pos': False,
 'valid_batch_size': 16}


In [3]:
import numpy as np
import pandas as pd

from joblib import Parallel, delayed
#from tqdm import tqdm
from tqdm import tqdm_notebook as tqdm
import spacy
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold

import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
import pickle
import gc

In [4]:
if 'google.colab' in sys.modules:
    use_cosine = CFG["use_cosine"]
    is_single = CFG["is_single"]
    debug = CFG["debug"]
elif 'kaggle_web_client' in sys.modules:
    use_cosine = False
    is_single = False
    debug = False

In [5]:
df = pd.read_csv("../input/coleridgeinitiative-show-us-the-data/train.csv")
labels = np.sort(df["cleaned_label"].unique())
labels = list(labels)

if debug:
    labels = labels[:20]
    print("Debug with the length of", len(labels))

***
# Cosine Similarity Operation

In [58]:
# labels2manualcat
labels2manualcat = {
    '2019 ncov complete genome sequences':'2019 ncov genome sequences',
    '2019 ncov genome sequence':'2019 ncov genome sequences',
    '2019 ncov genome sequences':'2019 ncov genome sequences',
    'adni':'alzheimers disease neuroimaging initiative',
    'advanced national seismic system anss comprehensive catalog comcat ':'seismic system comprehensive catalog',
    'advanced national seismic system comprehensive catalog':'seismic system comprehensive catalog',
    'aging integrated database':'aging integrated database',
    'aging integrated database agid ':'aging integrated database',
    'agricultural resource management survey':'agricultural resources management survey',
    'agricultural resources management survey':'agricultural resources management survey',
    'alzheimer s disease neuroimaging initiative adni ':'alzheimers disease neuroimaging initiative',
    'alzheimers disease neuroimaging initiative':'alzheimers disease neuroimaging initiative',
    'anss comprehensive catalog':'seismic system comprehensive catalog',
    'anss comprehensive earthquake catalog':'seismic system comprehensive catalog',
    'arms farm financial and crop production practices':'arms farm financial and crop production practices',
    'baccalaureate and beyond':'baccalaureate and beyond longitudinal study',
    'baccalaureate and beyond longitudinal study':'baccalaureate and beyond longitudinal study',
    'baltimore longitudinal study of aging':'baltimore longitudinal study of aging',
    'baltimore longitudinal study of aging blsa ':'baltimore longitudinal study of aging',
    'beginning postsecondary student',
    'beginning postsecondary students',
    'beginning postsecondary students longitudinal study',
    'cas covid 19 antiviral candidate compounds data',
    'cas covid 19 antiviral candidate compounds data set',
    'cas covid 19 antiviral candidate compounds dataset',
    'cccsl',
    'census of agriculture',
    'characterizing health associated risks and your baseline disease in sars cov 2',
    'characterizing health associated risks and your baseline disease in sars cov 2 charybdis ',
    'coastal change analysis program',
    'coastal change analysis program land cover',
    'common core of data',
    'complexity science hub covid 19 control strategies list',
    'complexity science hub covid 19 control strategies list cccsl ',
    'covid 19 death data',
    'covid 19 genome sequence',
    'covid 19 genome sequences',
    'covid 19 image data collection',
    'covid 19 open research data',
    'covid 19 open research dataset',
    'covid 19 open research dataset cord 19 ',
    'covid open research dataset',
    'early childhood longitudinal study',
    'education longitudinal study',
    'ffrdc research and development survey',
    'genome sequence of 2019 ncov',
    'genome sequence of covid 19',
    'genome sequence of sars cov 2',
    'genome sequences of 2019 ncov',
    'genome sequences of covid 19',
    'genome sequences of sars cov 2',
    'high school longitudinal study',
    'higher education research and development survey',
    'ibtracs',
    'international best track archive for climate stewardship',
    'jh crown registry',
    'nass census of agriculture',
    'national assessment of education progress',
    'national center for education statistics common core of data',
    'national center for science and engineering statistics higher education research and development survey',
    'national center for science and engineering statistics survey of doctorate recipients',
    'national center for science and engineering statistics survey of earned doctorates',
    'national center for science and engineering statistics survey of graduate students and postdoctorates in science and engineering',
    'national center for science and engineering statistics survey of industrial research and development',
    'national center for science and engineering statistics survey of science and engineering research facilities',
    'national education longitudinal study',
    'national oceanic and atmospheric administration c cap',
    'national oceanic and atmospheric administration optimum interpolation sea surface temperature',
    'national oceanic and atmospheric administration world ocean database',
    'national science foundation higher education research and development survey',
    'national science foundation survey of doctorate recipients',
    'national science foundation survey of earned doctorates',
    'national science foundation survey of graduate students and postdoctorates in science and engineering',
    'national science foundation survey of industrial research and development',
    'national science foundation survey of science and engineering research facilities',
    'national teacher and principal survey',
    'national water level observation network',
    'national weather service nws storm surge risk',
    'nces common core of data',
    'ncses survey of doctorate recipients',
    'ncses survey of graduate students and postdoctorates in science and engineering',
    'noaa c cap',
    'noaa international best track archive for climate stewardship',
    'noaa national water level observation network',
    'noaa optimum interpolation sea surface temperature',
    'noaa sea lake and overland surges from hurricanes',
    'noaa storm surge inundation',
    'noaa tidal station',
    'noaa tide gauge',
    'noaa tide station',
    'noaa water level station',
    'noaa world ocean database',
    'north american breeding bird survey',
    'north american breeding bird survey bbs ',
    'nsf ffrdc research and development survey',
    'nsf higher education research and development survey',
    'nsf survey of earned doctorates',
    'nsf survey of graduate students and postdoctorates in science and engineering',
    'nsf survey of industrial research and development',
    'nsf survey of science and engineering research facilities',
    'optimum interpolation sea surface temperature',
    'our world in data',
    'our world in data covid 19',
    'our world in data covid 19 dataset',
    'program for the international assessment of adult competencies',
    'rsna international covid 19 open radiology database',
    'rsna international covid 19 open radiology database ricord ',
    'rsna international covid open radiology database',
    'rural urban continuum codes',
    'sars cov 2 complete genome sequence',
    'sars cov 2 complete genome sequences',
    'sars cov 2 full genome sequence',
    'sars cov 2 full genome sequences',
    'sars cov 2 genome sequence',
    'sars cov 2 genome sequences',
    'school survey on crime and safety',
    'sea surface temperature optimum interpolation',
    'slosh model',
    'survey of doctorate recipients',
    'survey of earned doctorates',
    'survey of graduate students and postdoctorates in science and engineering',
    'survey of industrial research and development',
    'survey of science and engineering research facilities',
    'survey of state government research and development',
    'the national institute on aging genetics of alzheimer s disease data storage site',
    'the national institute on aging genetics of alzheimer s disease data storage site niagads ',
    'trends in international mathematics and science study',
    'usda census of agriculture',
    'usgs north american breeding bird survey',
    'world ocean database'


}

['2019 ncov complete genome sequences',
 '2019 ncov genome sequence',
 '2019 ncov genome sequences',
 'adni',
 'advanced national seismic system anss comprehensive catalog comcat ',
 'advanced national seismic system comprehensive catalog',
 'aging integrated database',
 'aging integrated database agid ',
 'agricultural resource management survey',
 'agricultural resources management survey',
 'alzheimer s disease neuroimaging initiative adni ',
 'alzheimers disease neuroimaging initiative',
 'anss comprehensive catalog',
 'anss comprehensive earthquake catalog',
 'arms farm financial and crop production practices',
 'baccalaureate and beyond',
 'baccalaureate and beyond longitudinal study',
 'baltimore longitudinal study of aging',
 'baltimore longitudinal study of aging blsa ',
 'beginning postsecondary student',
 'beginning postsecondary students',
 'beginning postsecondary students longitudinal study',
 'cas covid 19 antiviral candidate compounds data',
 'cas covid 19 antiviral can

In [9]:
def cosim(doc1, doc2):
    """
    Args:
        doc1: str
        doc2: str
    Returns:
        sim: float
    """
    nlp = spacy.load("en_core_web_sm")  # make sure to use larger package!
    # reference: https://github.com/explosion/spaCy/issues/3552

    # Rule base
    if doc1 == '2019 ncov complete genome sequences':
        doc1 = '2019 ncov genome sequences'
    elif doc1 == 'adni':
        doc1 = 'alzheimers disease neuroimaging initiative'
    elif doc1 == 'high school longitudinal study':
        doc1 = 'higher education research and development survey'
    elif doc1 == 'ibtracs':
        doc1 = 'international best track archive for climate stewardship'
    elif doc1 == 
    
    doc1 = nlp(doc1)
    doc2 = nlp(doc2)
    
    sim = doc1.similarity(doc2)
    return sim

In [17]:
if use_cosine:
    if is_single:
        outs = []
        with tqdm(total=len(labels)*len(labels)) as pbar:
            for doc1 in tqdm(labels):
                for doc2 in tqdm(labels):
                    outs.append(cosim(doc1, doc2))
                    pbar.update(1)
    else:
        outs = Parallel(n_jobs=-1)(delayed(cosim)(doc1, doc2) for doc2 in tqdm(labels) for doc1 in tqdm(labels))

    outs = np.array(outs).reshape((len(labels), len(labels)))
    np.fill_diagonal(outs, 0)
    
    if 'google.colab' in sys.modules:
        pickle.dump(outs, open(f"./{dname}/outs.pkl", "wb"))
    elif 'kaggle_web_client' in sys.modules:
        pickle.dump(outs, open(f"./outs.pkl", "wb"))

    del outs
    gc.collect()


***
# Simple String Operation

In [7]:
if not use_cosine:
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
    
    # Lemmatization
    # reference: https://stackoverflow.com/questions/38763007/how-to-use-spacy-lemmatizer-to-get-a-word-into-basic-form/48948642
    lab2lem = dict()
    for lab in labels:
        doc = nlp(lab)
        lem = " ".join([token.lemma_ for token in doc])
        lab2lem[lab] = lem
        
    lemmas = [lab2lem[label] for label in labels]
    
    
    outs = []
    for doc1 in lemmas:
        for doc2 in lemmas:
            outs.append(doc1 in doc2)
    
    outs = np.array(outs).reshape((len(lemmas), len(lemmas)))
    np.fill_diagonal(outs, False)

    if 'google.colab' in sys.modules:
        pickle.dump(outs, open(f"./{dname}/outs.pkl", "wb"))
    elif 'kaggle_web_client' in sys.modules:
        pickle.dump(outs, open(f"./outs.pkl", "wb"))

    del outs
    gc.collect()

***
# Graph

In [8]:
if 'google.colab' in sys.modules:
    outs = pickle.load(open(f"./{dname}/outs.pkl", "rb"))
elif 'kaggle_web_client' in sys.modules:
    outs = pickle.load(open(f"./outs.pkl", "rb"))

In [9]:
if use_cosine:
    thres = 0.7
    for i,label in enumerate(labels):
        similars = labels[outs[i, :] > thres]
        print(f"{label}: {similars}")
        print()

In [10]:
if use_cosine:
    outs = outs > thres

In [11]:
# function for adding edge to graph
graph = defaultdict(list)
def addEdge(graph,u,v):
    graph[u].append(v)

# definition of function
def generate_edges(graph):
    edges = []
    # for each node in graph
    for node in graph:
        # for each neighbour node of a single node
        for neighbour in graph[node]:
            # if edge exists then append
            edges.append((node, neighbour))
    return edges

In [12]:
labels = np.array(labels)

for i in range(len(labels)):
    u = labels[i]
    vs = labels[outs[i, :]]
#     print("doc:\n", u)
#     print("Similar docs:\n", vs)
#     print()
    for v in vs:
        addEdge(graph, u, v)
        
edges = generate_edges(graph)

G = nx.Graph()
G.add_edges_from(edges)

In [ ]:
#nx.draw(G, with_labels=True)

In [ ]:
# for cc in nx.connected_components(G):
#     print(cc)

In [13]:
mycat = dict()
for label in labels:
    try:
        cat = "/".join(nx.node_connected_component(G, label))
        #print(f"label: {label},\n cat: {cat}\n")
    except KeyError:
        cat = label
    mycat[label] = cat

In [14]:
mycat

{'2019 ncov complete genome sequences': '2019 ncov complete genome sequences',
 '2019 ncov genome sequence': '2019 ncov genome sequence/2019 ncov genome sequences',
 '2019 ncov genome sequences': '2019 ncov genome sequences/2019 ncov genome sequence',
 'adni': 'adni/alzheimer s disease neuroimaging initiative adni ',
 'advanced national seismic system anss comprehensive catalog comcat ': 'anss comprehensive catalog/advanced national seismic system anss comprehensive catalog comcat ',
 'advanced national seismic system comprehensive catalog': 'advanced national seismic system comprehensive catalog',
 'aging integrated database': 'aging integrated database',
 'aging integrated database agid ': 'aging integrated database agid ',
 'agricultural resource management survey': 'agricultural resource management survey',
 'agricultural resources management survey': 'agricultural resources management survey',
 'alzheimer s disease neuroimaging initiative adni ': 'adni/alzheimer s disease neuroima

***
# CV

In [15]:
df["pub_category"] = df["cleaned_label"].apply(lambda x: mycat.get(x))
df["pub_category"].fillna("other", inplace=True)

In [16]:
def get_cv(dataset, num_splits=None, col_target=None, col_group=None):
    """
    Args:
        dataset: pd.DataFrame
        num_splits: int
        col_target: str
        col_group: str
    Returns:
        folds: pd.DataFrame
    """
    X = dataset.index.values
    y = dataset[col_target].values
    groups = dataset[col_group].values

    group_kfold = GroupKFold(n_splits=num_splits)
    group_kfold.get_n_splits(X, y, groups)
    
    folds = pd.DataFrame()
    for i, (_, test_index) in enumerate(group_kfold.split(X, y, groups)):
        X_test = X[test_index]
        X_test = dataset[dataset.index.isin(X_test)]
        
        # Concat all and save at once
        X_test["fold"] = i+1
        folds = pd.concat([folds, X_test], ignore_index=True)
    
    return folds

In [17]:
folds = get_cv(df, num_splits=5, col_target="cleaned_label", col_group="pub_category")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
for i in range(5):
    train = folds[folds["fold"] != i+1]
    dev = folds[folds["fold"] == i+1]
    print(f"CV: {i+1} -------------------------------------------------------------------")
    print()
    print("#### train pub_category:\n\n", train["pub_category"].unique())
    print()
    print("#### dev pub_category:\n\n", dev["pub_category"].unique())
    print()
    print()

CV: 1 -------------------------------------------------------------------

#### train pub_category:

 ['noaa tidal station' 'slosh model'
 'baltimore longitudinal study of aging blsa /baltimore longitudinal study of aging'
 'noaa sea lake and overland surges from hurricanes'
 'the national institute on aging genetics of alzheimer s disease data storage site/the national institute on aging genetics of alzheimer s disease data storage site niagads '
 'national center for science and engineering statistics survey of industrial research and development/nsf survey of industrial research and development/national science foundation survey of industrial research and development/survey of industrial research and development'
 'program for the international assessment of adult competencies'
 'optimum interpolation sea surface temperature/noaa optimum interpolation sea surface temperature/national oceanic and atmospheric administration optimum interpolation sea surface temperature'
 'coastal chan

***
# Get text & Export

In [22]:
def get_text(filename, train=False):
    if train:
        df = pd.read_json(f'../input/coleridgeinitiative-show-us-the-data/train/{filename}.json')
    else:
        df = pd.read_json(f'../input/coleridgeinitiative-show-us-the-data/test/{filename}.json')

    text = " ".join(list(df['text']))
    return text

if 'google.colab' in sys.modules:
    folds.to_pickle(f"./{dname}/folds_pubcat.pkl")
    print("get_text should be run on Kaggle Platform")
    print("Please consider to use kagglenb006-get-text with folds_pubcat.pkl as its input...")
    print("Starting to upload folds_pubcat.pkl to Kaggle Platform...")
    try:
        !kaggle datasets init -p {dname}
        with open(f"{dname}/dataset-metadata.json", "r") as jsonFile:
            data = json.load(jsonFile)
        data["id"] = f"riow1983/{dname}"
        data["title"] = dname
        with open(f"{dname}/dataset-metadata.json", "w") as jsonFile:
            json.dump(data, jsonFile)
        !kaggle datasets create -p {dname}
    except:
        !kaggle datasets version -p {dname} -m "[Update]"

elif 'kaggle_web_client' in sys.modules:
    folds['text'] = folds['Id'].apply(lambda x: get_text(x, train=True))
    folds.to_pickle("./folds_pubcat.pkl")

get_text should be run on Kaggle Platform
Please consider to use kagglenb006-get-text with folds_pubcat.pkl as its input...
Starting to upload folds_pubcat.pkl to Kaggle Platform...
Data package template written to: nb009-cv/dataset-metadata.json
Starting upload for file outs.pkl
100% 16.7k/16.7k [00:01<00:00, 13.0kB/s]
Upload successful: outs.pkl (17KB)
Starting upload for file folds_pubcat.pkl
100% 2.49M/2.49M [00:01<00:00, 2.16MB/s]
Upload successful: folds_pubcat.pkl (2MB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/riow1983/nb009-cv
